In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.6 MB/s eta 0:00:00


In [16]:
import pickle as pkl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
# import tensorflow
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import wordcloud
import matplotlib.pyplot as plt
import numpy as np
import re

In [3]:
"""
Mounting G-drive
"""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
"""
Loading dataset for analysis
"""

df = pd.read_csv(r"/content/drive/MyDrive/Airline_Sentiment_analysis/dataset/Usecase3_Customer_Sentiment_Dataset.csv")

In [5]:
df.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


In [6]:
def tweet_preprocessor(tweet):
    # precprcess tweet
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    return tweet_proc

In [7]:
df['text'] = df['text'].apply(lambda x: tweet_preprocessor(x))

In [8]:
# Encoded the target column
lb=LabelEncoder()
df['Label'] = lb.fit_transform(df['airline_sentiment'])

In [9]:
#Generating Embeddings using tokenizer
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [10]:
#Model Building
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 120)           60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 30, 120)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 704)               2323200   
                                                                 
 dense (Dense)               (None, 352)               248160    
                                                                 
 dense_1 (Dense)             (None, 3)                 1059      
                                                                 
Total params: 2,632,419
Trainable params: 2,632,419
Non-trainable params: 0
______________________________________________

In [11]:
#Splitting the data into training and testing
y=pd.get_dummies(df['Label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [12]:
#Model Training
batch_size=32
model.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')


Epoch 1/10
366/366 [==============================] - 46s 103ms/step - loss: 0.7065 - accuracy: 0.7071
Epoch 2/10
366/366 [==============================] - 29s 79ms/step - loss: 0.5686 - accuracy: 0.7719
Epoch 3/10
366/366 [==============================] - 27s 74ms/step - loss: 0.5368 - accuracy: 0.7812
Epoch 4/10
366/366 [==============================] - 29s 79ms/step - loss: 0.5186 - accuracy: 0.7947
Epoch 5/10
366/366 [==============================] - 27s 73ms/step - loss: 0.4951 - accuracy: 0.8010
Epoch 6/10
366/366 [==============================] - 28s 78ms/step - loss: 0.4822 - accuracy: 0.8024
Epoch 7/10
366/366 [==============================] - 26s 70ms/step - loss: 0.4711 - accuracy: 0.8091
Epoch 8/10
366/366 [==============================] - 28s 77ms/step - loss: 0.4556 - accuracy: 0.8128
Epoch 9/10
366/366 [==============================] - 27s 74ms/step - loss: 0.4428 - accuracy: 0.8185
Epoch 10/10
366/366 [==============================] - 29s 78ms/step - loss: 0.42

In [12]:
# Model Testing
print(r"NN test score on 20% split: ")
model.evaluate(X_test,y_test)

In [14]:
def predict_sentiment(tweet):
    # Tokenize and pad the input tweet
    tweet_seq = tokenizer.texts_to_sequences([tweet])
    tweet_padded = pad_sequences(tweet_seq, maxlen=X.shape[1], padding='post')
    
    # Make the prediction
    sentiment_probs = model.predict(tweet_padded)[0]
    
    # Map the predicted sentiment probabilities to the sentiment labels
    sentiment_labels = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiment_labels[np.argmax(sentiment_probs)]
    
    return predicted_sentiment

In [17]:
tweet = "This airline is the worst. I will never fly with them again."
predicted_sentiment = predict_sentiment(tweet)
print(predicted_sentiment)

1/1 [==============================] - 0s 26ms/step
negative


In [25]:
tweet = "I can fly with them again"
predicted_sentiment = predict_sentiment(tweet)
print(predicted_sentiment)

1/1 [==============================] - 0s 96ms/step
neutral
